# WORK IN PROGRESS

Not sure what widget options would be best and how to write a performant callback that selects and processes values for each timestep

In [11]:
import pyvista as pv
import numpy as np
import xarray as xr
from JulesD3D.utils import quickDF
from JulesD3D.processNetCDF import fixMeshGrid, addVectorSum, addDepth, makeVelocity, addUnderlayerCoords
from JulesD3D.plotPyVista import makeBottomSurface, makeStructuredGridDepth, easyTimeFormat
from cmocean.cm import deep
import matplotlib.pyplot as plt
from JulesD3D.utils import ncFilepicker
from os import path

In [86]:
folder = '/Users/julesblom/ThesisResults/'

nc_filename = ncFilepicker(folder)

In [87]:
nc_filename

Select(description='NetCDF file:', layout=Layout(height='120px', width='100%'), options=('/Users/julesblom/The…

In [88]:
basename = path.basename(nc_filename.value)
plot_title, _ = path.splitext(basename)
plot_title

'trim-36km_200m_W60ChannelRun01MoreOutput'

In [89]:
trim = xr.open_dataset(nc_filename.value) # Open with xarray : DataSet : A collection of DataArrays

In [90]:
if 'velocity' not in trim:
    trim = fixMeshGrid(trim, trim.XZ.values, trim.YZ.values, mystery_flag=True)
    trim = addDepth(trim)
#     trim = makeVelocity(trim)
    trim = addUnderlayerCoords(trim)
else:
    print("DataSet is preprocessed, 'velocity' is already in DataSet")

------ Fixing mesh grid, assuming a uniform grid ------
x_gridstep:	 200.0 m
y_gridstep:	 200.0 m
width:		 26000.0 m
length:		 36200.0 m
132 x 182 grid


### Process data vars

In [6]:
concentration = trim.R1.transpose('time', 'LSTSCI', 'M', 'N', 'KMAXOUT_RESTR')
# concentration_silt = concentration.isel(LSTSCI=1)
concentration_silt = concentration.isel(LSTSCI=1)
concentration_silt_at_time = concentration_silt.isel(time=5)
concentration_silt_at_time = concentration_silt_at_time.assign_attrs({'long_name': 'Silt concentrations percentage per layer in zeta point',
                                                              'units': '%',
                                                              'grid': 'grid',
                                                              'location': 'face'})

In [21]:
# # Reorder velocity dims to match x, y, z
# velocity = trim.velocity.transpose('time', 'M', 'N', 'KMAXOUT_RESTR')
# velocity.dims

In [91]:
density = trim.RHO.transpose('time', 'M', 'N', 'KMAXOUT_RESTR')
density.dims

('time', 'M', 'N', 'KMAXOUT_RESTR')

In [ ]:
# trim['RHO'] = trim['RHO'].where(trim['RHO'].values == 1025, 0)

### Turn Delft3D output into PyVista grid

In [92]:
bottom_surface = makeBottomSurface(trim, timestep=0)
bottom_surface

StructuredGrid (0x11d92cfa0)
  N Cells:	23400
  N Points:	23711
  X Bounds:	0.000e+00, 2.600e+04
  Y Bounds:	1.000e+02, 3.610e+04
  Z Bounds:	-6.999e+02, -0.000e+00
  Dimensions:	131, 181, 1
  N Arrays:	1

In [93]:
depth_centers_grid = makeStructuredGridDepth(trim, keyword='SIG_LYR')

'depth' DataArray already found in DataSet!
xyz_layers.shape (1921920, 3)


In [94]:
initial_timestep = 10
time_outputstep = trim.time.isel(time=initial_timestep).values
time_lims = (initial_timestep, trim.time.size-1)
time_lims

(10, 190)

In [67]:
sargs = dict(
    title_font_size=30,
    label_font_size=28,
    shadow=True,
    n_labels=6,
    fmt="%.1f",
    font_family="arial",
)

In [112]:
depth_centers_grid['Density'] = density.isel(time=initial_timestep).values.ravel()
density_threshold_lim = [1025.25, 1040]
threshed_density_centers = depth_centers_grid.threshold(density_threshold_lim, scalars="Density")

outline = depth_centers_grid.outline()
opacity_trick = [0] * 6 + [0.9] * 14

In [113]:
p = pv.Plotter(notebook=False)
p.add_mesh(outline)
# p.add_mesh(bottom_surface, cmap=deep)
p.add_mesh(depth_centers_grid, scalars='Density', clim=(1020, 1040), opacity=opacity_trick, scalar_bar_args=sargs)
initTimeActor = p.add_text(f"Density at {easyTimeFormat(time_outputstep)}", position='upper_left')
p.add_slider_widget(callback, time_lims, value=initial_timestep, event_type='end')
p.enable_depth_peeling()
p.show_grid()
p.set_scale(zscale=25)
panel = p.show()

2020-02-01T01:00:00.000000000
1084.23
2020-02-01T01:00:00.000000000
1036.041
2020-02-01T05:00:00.000000000
1111.2157
2020-02-01T02:06:00.000000000


In [110]:
def callback(val):
    p.remove_actor(initTimeActor)
    outputstep = int(val)
    Density = density.isel(time=outputstep).values.ravel()

    print(Density.max())

    p.update_scalars(Density, mesh=depth_centers_grid) 
    
    time_outputstep = trim.time.isel(time=outputstep).values
    timeActor = p.add_text(f"Density at {easyTimeFormat(time_outputstep)}", position='upper_left')

# Old approaches

In [ ]:
# alg = vtk.vtkThreshold()
# alg.SetInputDataObject(depth_centers_grid)
# alg.SetInputArrayToProcess(0, 0, 0, field, scalars) # args: (idx, port, connection, field, name)
# alg.SetUseContinuousCellRange(continuous)

In [ ]:
# def testCallback(val):
#     outputstep = int(val)
    
#     scalars = density.isel(time=outputstep).values.ravel()
#     print(scalars.max())
# #     concentration_silt.isel(time=outputstep).values.ravel()
#     p.update_scalars(scalars, mesh=depth_centers_grid)
# #     print("Updated scalars", scalars.size)
# #     alg.SetInputDataObject(depth_centers_grid)
# #     threshold_mesh = pv.wrap(alg.GetOutput())
    
# #     alg.ThresholdByLower(value)
# #     alg.update()

In [ ]:
# def oldCallback(val):
#     outputstep = int(val)
#     scalars = concentration_silt.isel(time=outputstep).values.ravel()
#     depth_centers_grid['conc_silt'] = scalars
    
#     thresh_conc_silt = depth_centers_grid.threshold(conc_silt_lims, scalars="conc_silt")

#     threshed_actor = p.add_mesh(thresh_conc_silt, scalars='conc_silt', cmap="plasma",
#                        opacity=0.8, clim=conc_silt_lims, ambient=0.2)
#     print("outputstep", outputstep)
    

In [ ]:
# p = pv.Plotter(notebook=False)
# p.add_mesh(bottom_surface, cmap=deep)
# p.add_mesh(depth_centers_grid, opacity=0.9)
# # p.add_mesh(thresh_conc_silt, scalars='conc_silt', cmap="plasma", opacity=0.8, clim=conc_silt_lims, ambient=0.2)
# # p.add_mesh_threshold(thresh_conc_silt, scalars='conc_silt', cmap="plasma", opacity=0.8, clim=conc_silt_lims, ambient=0.2)
# # p.add_slider_widget(testCallback, time_lims, value=5)
# p.add_slider_widget(testCallback, time_lims, value=5)
# p.enable_depth_peeling()
# p.show_grid()
# p.set_scale(zscale=25)
# panel = p.show()